# Recurrent Neural Networks
## CCN reminders

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

model.add(Conv2D(32, (5, 5), 
                 padding='same',
                 strides = (1,1),
                 input_shape=(32, 32, 3), 
                 activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(40, activation='relu'))
model.add(Dense(10, activation='softmax'))

2022-08-18 09:08:11.650550: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-18 09:08:11.654282: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-18 09:08:11.654292: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-18 09:08:13.871326: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-18 09:08:13.871362: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

## Transfer learning reminder

In [2]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# This is optional, depending if you have time & will to update the first layers
model.trainable = False

flatten_layer = layers.Flatten()
dense_layer = layers.Dense(100, activation='relu')
prediction_layer = layers.Dense(10, activation='softmax')

model = Sequential([
    model, 
    flatten_layer, 
    dense_layer, 
    prediction_layer
])

58889256/58889256 [==============================] - 24s 0us/step


## Architecture

In [3]:
import numpy as np

# --- SEQUENCE A (Paris)

day_1 = [10, 25, 50]  # OBSERVATION 1 [Temp, speed, pollution]
day_2 = [13, 10, 70]  # OBSERVATION 2 [Temp, speed, pollution]
day_3 = [ 9,  5, 90]
day_4 = [ 7,  0, 95]

sequence_a = [day_1, day_2, day_3, day_4]

y_a = 110 # Pollution day 5

# --- SEQUENCE B (Berlin)
sequence_b = [[25, 20, 30], [26, 24, 50], [28, 20, 80], [22, 3, 110]]
y_b = 125

# --- SEQUENCE C (London)
sequence_c = [[15, 10, 60], [25, 20, 65], [35, 10, 75], [36, 15, 70]]
y_c = 30

X = np.array([sequence_a, sequence_b, sequence_c]).astype(np.float32)
y = np.array([y_a, y_b, y_c]).astype(np.float32)

print(X.shape)
print(y.shape)

(3, 4, 3)
(3,)


In [5]:
# ––– It is very easy to code

from keras.api._v2.keras.models import Sequential
from keras.api._v2.keras import layers

model = Sequential()
model.add(layers.SimpleRNN(units=2, activation='tanh', input_shape=(4,3)))
model.add(layers.Dense(1, activation="linear"))

# The compilation
model.compile(loss='mse', 
              optimizer='rmsprop')  # Recommended optimizer for RNNs
# The fit
model.fit(X, y,
         batch_size=16,
         epochs=10, verbose=0)

# The prediction (one per sequence/city)
model.predict(X)

1/1 [==============================] - 0s 169ms/step


array([[-0.9255787 ],
       [-0.99157965],
       [-0.9916189 ]], dtype=float32)

In [7]:
n_x = 3 # features (temp, speed, pollution)
n_h = 2 # units = neurons

# number of weights
n_h * n_x + n_h * n_h + n_h

12

In [ ]:
# ---- NEW CITY ----
x = [[10, 25, 50],  # x1
     [13, 10, 70],  # x2
     [ 9,  5, 90],  # x3
     [ 7,  0, 95]]  # x4

y4 = # Pollution day 4+1 = 5

In [8]:
print("Input shape: ", X.shape)
model.summary()

Input shape:  (3, 4, 3)
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 2)                 12        
                                                                 
 dense_5 (Dense)             (None, 1)                 3         
                                                                 
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [11]:
[w.numpy().shape for w in model.layers[0].weights]

[(3, 2), (2, 2), (2,)]

In [12]:
# Take this model with 10 RNN units
model_10 = Sequential()
model_10.add(layers.SimpleRNN(units=10))
model_10.add(layers.Dense(1, activation="linear"))

model_10.compile(loss='mse', optimizer='rmsprop')
model_10.fit(X, y, batch_size=16, epochs=10, verbose=0)
model_10.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 10)                140       
                                                                 
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________


In [13]:
import numpy as np

# --- SEQUENCE A (Paris)

sequence_a = [[10, 25, 50],  # OBS day 1
              [13, 10, 70],  # OBS day 2
              [ 9,  5, 90],  # OBS day 3
              [ 7,  0, 95]]  # OBS day 4

y_a = [70,   # flu cases day 1
       90,   # flu cases day 2
       95,   # flu cases day 3
       110,] # flu cases day 4

# --- SEQUENCE B (Berlin)
sequence_b = [[25, 20, 30], [26, 24, 50], [28, 20, 80], [22, 3, 110]]
y_b = [50, 80, 110, 125]

# --- SEQUENCE C (London)
sequence_c = [[15, 10, 60], [25, 20, 65], [35, 10, 75], [36, 15, 70]]
y_c = [65, 75, 70, 30]

X = np.array([sequence_a, sequence_b, sequence_c]).astype(np.float32)
y = np.array([y_a, y_b, y_c]).astype(np.float32)

print(X.shape)
print(y.shape)

(3, 4, 3)
(3, 4)


In [14]:
model_2 = Sequential()

model_2.add(layers.SimpleRNN(2, return_sequences=True))
model_2.add(layers.Dense(1, activation='relu'))

model_2.compile(loss='mse', optimizer='rmsprop')
model_2.fit(X, y, verbose=0)

In [15]:
print("y_pred shape:", model_2.predict(X).shape)

1/1 [==============================] - 0s 177ms/step
y_pred shape: (3, 4, 1)


## Stacking RNNs

In [16]:
model_stacked = Sequential()

model_stacked.add(layers.SimpleRNN(10, return_sequences=True, input_shape=((4, 3)))) 
model_stacked.add(layers.SimpleRNN(3, return_sequences=False))
model_stacked.add(Dense(1, activation='relu'))

model_stacked.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, 4, 10)             140       
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 3)                 42        
                                                                 
 dense_8 (Dense)             (None, 1)                 4         
                                                                 
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________


## Zoology

In [17]:
from keras.api._v2.keras.layers import SimpleRNN, LSTM, GRU

###  Simple RNN  ###
model = Sequential()
model.add(SimpleRNN(units=10, activation='tanh'))  

###  LSTM   ###
model = Sequential()
model.add(LSTM(units=10, activation='tanh'))

###  GRU ###
model = Sequential()
model.add(GRU(units=10, activation='tanh'))

# Compile with 'rmsprop' rather than 'adam' (recommended)
model.compile(loss='mse',
              optimizer='rmsprop') 

## Inputs of different lengths

In [18]:
import numpy as np

# --- SEQUENCE 1 (Paris) ---

sequence_1 = [[10, 25, 50],  # OBS day 1
              [13, 10, 70],  # OBS day 2
              [ 9,  5, 90],  # OBS day 3
              [ 7,  0, 95]]  # OBS day 4

y_1 = 110 # pollution day 5 

# --- SEQUENCE 2 (Berlin) ---
sequence_2 = [[25, 20, 30],
              [26, 24, 50]]

y_2 = 125 # pollution day 3

# --- SEQUENCE 3 (London)
sequence_3 = [[15, 10, 60],
              [25, 20, 65],
              [35, 10, 75]]
y_3 = 30 # Pollution day 4

X = [sequence_1, sequence_2, sequence_3]
X = [np.array(_) for _ in X]
y = np.array([y_1, y_2, y_3]).astype(np.float32)

In [19]:
model = Sequential()
model.add(SimpleRNN(1, activation='tanh')) 
model.add(Dense(1, activation="relu"))

# The compilation
model.compile(loss='mse', optimizer='rmsprop')

# The fit
model.fit(X, y, batch_size=16, epochs=10)

ValueError: Data cardinality is ambiguous:
  x sizes: 4, 2, 3
  y sizes: 3
Make sure all arrays contain the same number of samples.

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_pad = pad_sequences(X, dtype='float32') # int32 by default
X_pad

array([[[10., 25., 50.],
        [13., 10., 70.],
        [ 9.,  5., 90.],
        [ 7.,  0., 95.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [25., 20., 30.],
        [26., 24., 50.]],

       [[ 0.,  0.,  0.],
        [15., 10., 60.],
        [25., 20., 65.],
        [35., 10., 75.]]], dtype=float32)

### Padding

In [21]:
X_pad = pad_sequences(X, dtype='float32', padding='post', value=-1000)
X_pad

array([[[   10.,    25.,    50.],
        [   13.,    10.,    70.],
        [    9.,     5.,    90.],
        [    7.,     0.,    95.]],

       [[   25.,    20.,    30.],
        [   26.,    24.,    50.],
        [-1000., -1000., -1000.],
        [-1000., -1000., -1000.]],

       [[   15.,    10.,    60.],
        [   25.,    20.,    65.],
        [   35.,    10.,    75.],
        [-1000., -1000., -1000.]]], dtype=float32)

### Masking

In [22]:
from tensorflow.keras.layers import Masking

# –– Data
X = pad_sequences(X, dtype='float32', padding='post', value=-1000)
# X.shape == (3,4,3)

# –– Model
model = Sequential()
model.add(layers.Masking(mask_value=-1000, input_shape=(4,3)))
model.add(layers.SimpleRNN(units=2, activation='tanh'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

# –– Compilation
model.compile(loss='mse', 
              optimizer='rmsprop') # Use `rmsprop`

# –– Fit
model.fit(X, y);

1/1 [==============================] - 1s 812ms/step - loss: 9532.3359
